#Implementação da arquitetura

In [ ]:
import tensorflow as tf
from tensorflow.keras import  Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import MeanIoU
from keras import backend as K

In [ ]:
# Bloco convolucional utilizado em todas as etapas
def ConvBlock(tensor, num_feature):

  x = tf.keras.layers.Conv2D(num_feature, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(tensor)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Conv2D(num_feature, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  return x

def ponte(tensor, num_feature):
  x = ConvBlock(tensor,num_feature)
  return x

In [ ]:
# Seção encoder
def encoder_block(tensor, num_feature):
  x = ConvBlock(tensor, num_feature)
  p = tf.keras.layers.MaxPooling2D((2,2))(x)
  return x, p

#seção decoder
def decoder_block(tensor,skip_connection, num_feature):
  x = tf.keras.layers.Conv2DTranspose(num_feature, (2,2), strides=(2,2), padding='same')(tensor) #recebe do bloco anterior e faz upsampling
  x = tf.keras.layers.concatenate([x, skip_connection])
  x = ConvBlock(x, num_feature)
  return x


### estrutura da u-net

In [ ]:

def Unet(n_classes, tensor_shape):

  input = tf.keras.layers.Input(tensor_shape) #instancia o tensor para os dados de entrada

  #seção de contração:
  skip1, c1 = encoder_block(input,16) # 128x128x3 > 64x64x16
  skip2, c2 = encoder_block(c1,32) # 64x64x16 > 32x32x32
  skip3, c3 = encoder_block(c2,64) # 32x32x32 > 16x16x64
  skip4, c4 = encoder_block(c3,128) # 16x16x64 > 8x8x64

  #bottleneck
  c5 = ponte(c4,256) # 8x8x64 > 8x8x256

  #seção de expansão:
  c6 = decoder_block(c5, skip4, 128) #8x8x256 > 16x16x128
  c7 = decoder_block(c6, skip3, 64) #16x16x128 > 32x32x64
  c8 = decoder_block(c7, skip2, 32) #32x32x64 > 64x64x32
  c9 = decoder_block(c8, skip1, 16) #64x64x32 > 128x128x16

  #camada de saída:
  output = tf.keras.layers.Conv2D(n_classes, (1,1), activation='softmax')(c9) #128x128x16 > 128x128x8, 8= número de classes

  model = Model(input, output, name="U-Net")
  return model


### compilando

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as K

# Definição da função de perda Jaccard
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))

    Jaccard Distance Loss é útil para datasets desbalanceados.
    """
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    intersection = tf.reduce_sum(tf.abs(y_true * y_pred), axis=-1)
    sum_ = tf.reduce_sum(tf.abs(y_true) + tf.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth


# Testar a função com exemplos
y_pred = np.array([np.arange(-10, 10 + 0.1, 0.1)]).T
y_true = np.zeros(y_pred.shape)

# Plotar o Jaccard Distance Loss
try:
    loss = jaccard_distance_loss(y_true, y_pred).numpy()
except Exception as e:
    print("Erro ao calcular o Jaccard Distance Loss:", e)
else:
    plt.title("Jaccard Distance Loss")
    plt.plot(y_pred, loss)
    plt.show()

# Testar com casos específicos
y_true = np.array([[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1.0, 0]])
y_pred = np.array([[0, 0, 0.9, 0], [0, 0, 0.1, 0], [1, 1, 0.1, 1]])

loss = jaccard_distance_loss(y_true, y_pred).numpy()
print("Jaccard Distance Loss:", loss)
assert loss[0] < loss[1]
assert loss[1] < loss[2]

# Testar com Binary Cross-Entropy
binary_crossentropy = tf.keras.losses.binary_crossentropy(y_true, y_pred)
print("Binary Cross-Entropy:", binary_crossentropy.numpy())

In [ ]:
### Monitoramentto de IOU no treino

class IoUCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.validation_data[0])
        y_pred_argmax = np.argmax(y_pred, axis=-1).astype(np.int32)
        iou = MeanIoU(num_classes=num_classes)
        iou.update_state(self.validation_data[1], y_pred_argmax)
        print(f"Epoch {epoch + 1} - Mean IoU: {iou.result().numpy()}")

In [ ]:
#parâmetros:
input_shape = (256,256, 1)
num_classes = 9
lr = 0.003
#metrics = ["accuracy",MeanIoU(num_classes=num_classes)]
metrics = ["accuracy"]

#loss =  SparseCategoricalCrossentropy()
#loss = 'categorical_crossentropy'
loss = jaccard_distance_loss(y_test, y_pred).numpy()

model = Unet(num_classes, input_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=loss, metrics=metrics)
model.summary()

arquivo_modelo = '' # .h não é mais aceito
arquivo_modelo_json = ''


Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256, 256, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 256, 256, 16)   │            160 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_18    │ (None, 256, 256, 16)   │             64 │ conv2d_19[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 256, 256, 16)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 256, 256, 16)   │          2,320 │ dropout_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_19    │ (None, 256, 256, 16)   │             64 │ conv2d_20[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 128, 128, 16)   │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_21 (Conv2D)        │ (None, 128, 128, 32)   │          4,640 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_20    │ (None, 128, 128, 32)   │            128 │ conv2d_21[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 128, 128, 32)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_22 (Conv2D)        │ (None, 128, 128, 32)   │          9,248 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_21    │ (None, 128, 128, 32)   │            128 │ conv2d_22[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 64, 64, 32)     │              0 │ batch_normalization_2… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_23 (Conv2D)        │ (None, 64, 64, 64)     │         18,496 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_22    │ (None, 64, 64, 64)     │            256 │ conv2d_23[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout) 

 Total params: 1,946,841 (7.43 MB)

 Trainable params: 1,943,897 (7.42 MB)

 Non-trainable params: 2,944 (11.50 KB)

# Pré Processamento

## Utils


In [ ]:
import tensorflow as tf
import keras
import os
import re
import glob
import numpy as np
import cv2

**Dicionário RGB > Rótulo**

In [ ]:
def gen_dict_cores():
    mask_dict = {
        (0, 0, 0): 0,        # Preto = None
        (100, 0, 100): 1,    # Violeta = Liver (fígado)
        (255, 255, 255): 2,  # Branco = Bone (Osso)
        (0, 255, 0): 3,      # Verde = Gallbladder (Vesícula biliar)
        (255, 255, 0): 4,    # Amarelo = Kidney (Rins)
        (0, 0, 255): 5,      # Azul = Pancreas
        (255, 0, 0): 6,      # Vermelho = Vessels (Veias)
        (255, 0, 255): 7,    # Rosa = Spleen (Baço)
        (0, 255, 255): 8     # Azul claro = Adrenal (Glândula Adrenal)
    }
    return mask_dict

In [ ]:
""" Função que mapeia cada pixel da imagem ao valor rgb definido no dicionário,
 e substitui o valor pelo rótulo correspondente, em um novo array."""

def RGBtoClass(rgb, dictCores):
    arr = np.zeros(rgb.shape[:2])  # Inicializa a matriz de rótulos

    for color, label in dictCores.items():  # Itera sobre os pares (cor, rótulo)
        color = np.array(color)  # Converte a cor para um array NumPy
        arr[np.all(rgb == color, axis=-1)] = label  # Atribui o rótulo aos pixels que correspondem à cor

    return arr

In [ ]:
def onehot_to_rgb(oneHot, dictCores):
    oneHot = np.array(oneHot)  # Converte para array numpy
    oneHot = np.argmax(oneHot, axis=-1)  # Seleciona o maior valor (índice)
    output = np.zeros(oneHot.shape + (3,))  # Cria a matriz RGB de saída
    oneHot = np.expand_dims(oneHot, axis=-1)  # Expande as dimensões

    for color, index in dictCores.items():
        output[np.all(oneHot == index, axis=-1)] = color

    return np.uint8(output)

In [ ]:
"""esta função segmenta o nome do arquivo para o img_loader ordenar o dataset
na ordem do diretório e ter correspondência entre a lista de imagens e máscaras"""

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

""" img_loader recebe um caminho de diretório,uma lista vazia e  uma tupla com
dimensões de imagem. Lê as imagens png ou jpg do diretório, ordena pelo nome e
 armazena a imagem na lista img_data e seu nome na lista img_names"""
def img_loader(path, img_data, size=None, rgb=True):
  #lista para o nome dos arquivos
  img_names = []

  for diretorio_path in sorted(glob.glob(path)):
    for img_path in sorted(glob.glob(os.path.join(diretorio_path, "*.[pj]*[np]*[g]*")), key=natural_sort_key): #percorre o diretório na ordem natural dos títulos de arquivo
      img = cv2.imread(img_path,
                       cv2.IMREAD_COLOR if rgb
                       else cv2.IMREAD_GRAYSCALE) #img tem 3 canais na 3 dimensao se RGB, e 1 canal se preto/branco

      if rgb:  # Corrige para formato RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      if size is not None:
        img = cv2.resize(img, size) #redimensiona conforme o parâmetro

      img_data.append(img.astype(np.uint8)) #add a imagem na lista do parametro
      img_names.append(os.path.basename(img_path)) #add o nome do arquivo na lista de nomes

  #return img_data, img_names
  return np.array(img_data), img_names

In [ ]:
def save_dataset(X_train, X_val, X_test, y_train, y_val, y_test, save_dir="dataset"):
    os.makedirs(save_dir, exist_ok=True)

    np.save(os.path.join(save_dir, "X_train.npy"), X_train)
    np.save(os.path.join(save_dir, "X_val.npy"), X_val)
    np.save(os.path.join(save_dir, "X_test.npy"), X_test)
    np.save(os.path.join(save_dir, "y_train.npy"), y_train)
    np.save(os.path.join(save_dir, "y_val.npy"), y_val)
    np.save(os.path.join(save_dir, "y_test.npy"), y_test)

    print(f"Dataset salvo em {save_dir}")

def load_dataset(save_dir="dataset"):

    X_train = np.load(os.path.join(save_dir, "X_train.npy"))
    X_val = np.load(os.path.join(save_dir, "X_val.npy"))
    X_test = np.load(os.path.join(save_dir, "X_test.npy"))
    y_train = np.load(os.path.join(save_dir, "y_train.npy"))
    y_val = np.load(os.path.join(save_dir, "y_val.npy"))
    y_test = np.load(os.path.join(save_dir, "y_test.npy"))

    print(f"Dataset carregado de {save_dir}")
    return X_train, X_val, X_test, y_train, y_val, y_test


##Dataloader

In [ ]:
import tensorflow as tf
import keras
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split

In [ ]:
# caminhos para os diretórios das imagens
#real_img_treino = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Dataset_UltrassomAbdominal/abdominal_US/abdominal_US/RUS/images/train"
# não tem mascara de treino pra ultrasons reais
sim_img_treino = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Data/abdominal_US/AUS/images/train"
sim_mask_treino = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Data/abdominal_US/AUS/annotations/train"

#real_img_val = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Dataset_UltrassomAbdominal/abdominal_US/abdominal_US/RUS/images/test"
#real_mask_val = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Dataset_UltrassomAbdominal/abdominal_US/abdominal_US/RUS/annotations/test"
sim_img_val = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Data/abdominal_US/AUS/images/test"
sim_mask_val = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Data/abdominal_US/AUS/annotations/test"


In [ ]:
#conjuntos de treino
  #imagens simuladas
Simg_treino = []
Simg_treino_names = []
Smask_treino = []
Smask_treino_names = []
  #imagens reais
"""Rimg_treino = []
Rimg_treino_names = []
#ñ tem mascara real p teste"""

#conjuntos de teste
  #imagens simuladas
Simg_val = []
Simg_val_names = []
Smask_val = []
Smask_val_names = []

  #imagens reais
"""Rimg_val = []
Rimg_val_names = []
Rmask_val = []
Rmask_val_names = []"""


In [ ]:
#Carregando os dados:

dim = (256,256)
Simg_treino, Simg_treino_names = img_loader(sim_img_treino, Simg_treino,dim, False)
Smask_treino, Smask_treino_names = img_loader(sim_mask_treino,Smask_treino,dim)
#Rimg_treino, Rimg_treino_names = img_loader(real_img_treino,Rimg_treino,None,False)

Simg_val, Simg_val_names = img_loader(sim_img_val, Simg_val,dim, False)
Smask_val, Smask_val_names = img_loader(sim_mask_val,Smask_val,dim)
#Rimg_val, Rimg_val_names = img_loader(real_img_val, Rimg_val,None,False)
#Rmask_val, Rmask_val_names = img_loader(real_mask_val,Rmask_val)


In [ ]:
'''concatenando as imagens e mascaras de treino / teste para separar ao carregar
no  train_test_split'''
all_imgs = np.concatenate((Simg_treino, Simg_val), axis=0)
all_masks = np.concatenate((Smask_treino, Smask_val), axis=0)

all_imgname = Simg_treino_names + Simg_val_names
all_maskname = Smask_treino_names + Smask_val_names

all_imgs  = np.expand_dims(all_imgs, axis=3)
#all_imgs.shape, all_masks.shape

**Convertendo máscaras**

In [ ]:
dictCores = gen_dict_cores()
all_mask_class = []

for mask in  all_masks:
    onehotmask = RGBtoClass(mask, dictCores)
    all_mask_class.append(onehotmask)

all_mask_class = np.array(all_mask_class)
all_mask_class = np.expand_dims(all_mask_class, axis=3)

print("Classes únicas nos pixels das  máscaras :", np.unique(all_mask_class), all_mask_class.shape)

**Divisão do dataset em conjuntos de treino , validação e teste**



In [ ]:
# Divisão inicial: 10% para teste e 90% para treino e validação
X_train_val, X_test, y_train_val, y_test = train_test_split(
    all_imgs, all_mask_class, test_size=0.1, random_state=0, shuffle=False
)

# Segunda divisão: 15% para validação e 85% para teste
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.15, random_state=0, shuffle=False
)

# tamanhos dos conjuntos
print(f"Treino: {len(X_train)}, Validação: {len(X_val)}, Teste: {len(X_test)}")

In [ ]:
#normalizando os conjuntos
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

y_train = y_train.astype(np.int32)
y_val = y_val.astype(np.int32)
y_test = y_test.astype(np.int32)

In [ ]:
# Salvando os dados
save_dataset(X_train, X_val, X_test, y_train, y_val, y_test, save_dir = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal")

retorna X_train,

### **salvando o dataset**
 pq o collab ta colapsando de tanta coisa na RAM

In [ ]:
from sklearn.utils import class_weight

class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weights = {i: class_weights_array[i] for i in range(len(class_weights_array))}

KeyboardInterrupt: 

# Treinamento

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Carregar os dados
X_train, X_val, X_test, y_train, y_val, y_test = load_dataset(save_dir="/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Data")

Dataset carregado de /content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Data


In [ ]:
y_train_cat = to_categorical(y_train, num_classes = 9)
y_val_cat = to_categorical(y_val, num_classes = 9)

In [ ]:

y_test_cat = to_categorical(y_val, num_classes = 9)

In [ ]:
y_val_cat.shape

In [ ]:
## Data augmentation  ##########################################################

image_gen = ImageDataGenerator(
    rotation_range=15,            # Rotação aleatória de até 15 graus
    width_shift_range=0.3,        # Translação horizontal de até 30%
    height_shift_range=0.3,       # Translação vertical de até 30%
    shear_range=5,                # Cisalhamento de até 5%
    horizontal_flip=True,         # Flip horizontal
    vertical_flip=True,           # Flip vertical
    fill_mode='nearest',          # Preenchimento de regiões em branco após transformação
    brightness_range=[0.9, 1.1],  # Ajuste aleatório de brilho (80% a 120% do valor original)
    )

'''Mesmas transformações para as máscaras, exceto o brilho
para não afetar os valores das classes'''

mask_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=5,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
    )

#ajustando os geradores para as imagens
image_gen.fit(X_train, augment=True)
mask_gen.fit(y_train, augment=True)

# Instanciando os geradores para mascaras e imagens
image_generator = image_gen.flow(X_train, batch_size=32, seed=42)
mask_generator = mask_gen.flow(y_train, batch_size=32
, seed=42)

# Combinando os geradores para gerar lotes com as mesmas transformações
train_generator = zip(image_generator, mask_generator)
def augmenteixons():
    for img, mask in train_generator:
        yield img, mask

In [ ]:
## treinamento sem augmentation ################################################

#Parâmetros p/ callbacks
model_name = 'Jaccard01'
arquivo_modelo = f'/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal//Modelos/modelo_{model_name}.keras' # .h não é mais aceito
arquivo_modelo_json = f'/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal//Modelos/modelo_{model_name}.json'

#Callbacks
lr_reducer = ReduceLROnPlateau( monitor='val_loss', factor = 0.9, patience = 3, verbose = 1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience = 15, verbose = 1, mode='auto')
checkpointer = ModelCheckpoint(arquivo_modelo, monitor = 'val_loss', verbose = 1, save_best_only=True)


history = model.fit(X_train, y_train_cat,
                    batch_size=16,
                    verbose=1,
                    epochs = 100,
                    validation_data=(X_val, y_val_cat),
                    callbacks=[lr_reducer, early_stopper, checkpointer,IoUCallback()],
                    shuffle=False)


Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 694ms/step - accuracy: 0.5474 - loss: 1.1423
Epoch 1: val_loss improved from inf to 1.73513, saving model to /content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal//Modelos/modelo_Jaccard01.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 88s 961ms/step - accuracy: 0.5501 - loss: 1.1356 - val_accuracy: 0.1151 - val_loss: 1.7351 - learning_rate: 0.0030
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.8425 - loss: 0.3885
Epoch 2: val_loss did not improve from 1.73513
45/45 ━━━━━━━━━━━━━━━━━━━━ 13s 167ms/step - accuracy: 0.8426 - loss: 0.3879 - val_accuracy: 0.1151 - val_loss: 1.7351 - learning_rate: 0.0030
Epoch 3/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.8887 - loss: 0.2462
Epoch 3: val_loss did not improve from 1.73513
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 161ms/step - accuracy: 0.8883 - loss: 0.2468 - val_accuracy: 0.1151 - val_loss: 1.7351 - learning_rate: 0.0030
Epoch 4/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accu

In [ ]:
## Treinamento com augmentation ################################################
'''
model_name = 'Aug02'
arquivo_modelo = f'/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/modelo_{model_name}.keras' # .h não é mais aceito
arquivo_modelo_json = f'/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/modelo_{model_name}.json'


history = model.fit(augmenteixons(),
                    steps_per_epoch=len(X_train) // 32,
                    verbose=1,
                    epochs = 100,
                    validation_data=(X_val, y_val),
                    callbacks=[ checkpointer],
                    shuffle=False) '''

##Avaliação

In [ ]:
_, acc = model.evaluate(X_test, y_test)
print("Accuracy is = ", (acc * 100.0), "%")

##Plota a loss e accuracy de treino e validação a acda época
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


# Testes

In [ ]:
###### Carregando modelo #######

model_name = 'Aug02'
model = load_model(f'/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/modelo_{model_name}.keras')

In [ ]:
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
#predição do conjunto de teste
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

In [ ]:
####### avaliando pela metrica "intersection over union" no conjunto de teste ##
from keras.metrics import MeanIoU
n_classes = 9
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)

print("Mean IoU TESTE =", IOU_keras.result().numpy())


**Salvando todas as predições:**

In [ ]:
###### Salvando todas as predições #######

output_dir = "/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Testes/Teste5"
os.makedirs(output_dir, exist_ok=True)  # Cria o diretório, se não existir
dictCores = gen_dict_cores()

# Itera sobre todo o conjunto de teste
for idx in range(len(X_test)):
    # Seleciona a imagem, a máscara e a predição correspondentes
    img = X_test[idx]
    mask = y_test[idx]
    pred = y_pred[idx]

    # Cria o plot
    plt.figure(figsize=(12, 4))

    # Imagem original
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title("Imagem Original")
    plt.axis('off')

    # Máscara
    plt.subplot(1, 3, 2)
    plt.imshow(onehot_to_rgb(to_categorical(mask, num_classes=9), dictCores))
    plt.title("Ground Truth")
    plt.axis('off')

    # Predição
    plt.subplot(1, 3, 3)
    plt.imshow(onehot_to_rgb(pred, dictCores), cmap='gray')
    plt.title("Predição")
    plt.axis('off')

    # Salva o plot no diretório
    output_path = os.path.join(output_dir, f"plot_{idx}.png")
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

print(f"Todas as imagens foram salvas em: {output_dir}")

# Main

In [ ]:

import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import gradio as gr
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/Colab Notebooks/Segementação_Abdominal/Modelos/modelo_01_UnetUS.keras')

def gen_dict_cores_labels():
    mask_dict = {
        (0, 0, 0): (0, "Fundo"),             # Preto = None
        (100, 0, 100): (1, "Figado"),        # Violeta = Liver (fígado)
        (255, 255, 255): (2, "Osso"),       # Branco = Bone (Osso)
        (0, 255, 0): (3, "Vesicula biliar"),    # Verde = Gallbladder (Vesícula biliar)
        (255, 255, 0): (4, "Rim"),       # Amarelo = Kidney (Rins)
        (0, 0, 255): (5, "Pancreas"),       # Azul = Pancreas
        (255, 0, 0): (6, "Veia"),        # Vermelho = Vessels (Veias)
        (255, 0, 255): (7, "Baço"),       # Rosa = Spleen (Baço)
        (0, 255, 255): (8, "Gland. Adrenal")       # Azul claro = Adrenal (Glândula Adrenal)
    }
    return mask_dict

# Função para converter de one-hot para RGB e identificar classes
def onehot_to_rgb_labels(oneHot, dictCoresLabels):
    oneHot = np.array(oneHot)
    oneHot = np.argmax(oneHot, axis=-1)
    output = np.zeros(oneHot.shape + (3,))  # Cria a matriz RGB de saída
    present_classes = set()

    for color, (_, label) in dictCoresLabels.items():
        mask = oneHot == _ # Ajustado para evitar erro
        output[mask] = color
        if np.any(mask):
            present_classes.add(label)

    return np.uint8(output), list(present_classes)

# Função para criar legenda em HTML com nomes e cores
def create_legend_html(present_labels, dictCoresLabels):
    legend_html = "<ul style='list-style-type:none; padding:0;'>"
    for color, (_, label) in dictCoresLabels.items():
        if label in present_labels:
            color_hex = f"rgb{color}"  # Converte a cor para o formato RGB
            legend_html += f"<li style='color:{color_hex}; font-weight:bold;'>{label}</li>"
    legend_html += "</ul>"
    return legend_html

def predictmask_legend(image):
    dictCoresLabels = gen_dict_cores_labels()

    original_size = image.size
    image_resized = preprocess_image(image)

    y_pred = model.predict(image_resized)
    y_pred = y_pred[0]

    # Converte a predição para RGB e identifica os rótulos
    y_pred_rgb, labels = onehot_to_rgb_labels(y_pred, dictCoresLabels)

    # Redimensiona a máscara para o tamanho original da imagem
    y_pred_resized = cv2.resize(y_pred_rgb, original_size, interpolation=cv2.INTER_NEAREST)
    image_original_rgb = image.convert("RGB")

    # Combina a imagem original com a máscara
    alpha = 0.5
    image_combined = cv2.addWeighted(np.array(image_original_rgb), 1 - alpha, y_pred_resized, alpha, 0)
    legend_html = create_legend_html(labels, dictCoresLabels) #gera legenda

    return Image.fromarray(np.uint8(image_combined)), legend_html

def preprocess_image(image):

    image_resized = cv2.resize(np.array(image), (256, 256))
    image_resized = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # Normaliza e add batchdim
    image_resized = image_resized.astype('float32') / 255.0
    image_resized = np.expand_dims(image_resized, axis=0)

    # Adiciona uma dimensão extra p/ canal de cor
    image_resized = np.expand_dims(image_resized, axis=-1)

    return image_resized

# Interface Gradio
interface = gr.Interface(
    fn=predictmask_legend,   # Função para predição e legendas
    inputs=gr.Image(type="pil", label="Carregue uma imagem"),  # Entrada de imagem
    outputs=[
        gr.Image(type="pil", label="Imagem com Predição Sobreposta"),  # Saída da imagem com sobreposição
        gr.HTML(label="Legenda dos Órgãos Detectados")                # Saída em HTML para legenda colorida
    ],
    title="Segmentador de Ultrassom Abdominal",         # Título da interface
    description="Carregue uma ultrassonografia de cavidade abdominal para identificar os órgãos."
)

interface.launch()

In [ ]:
!pip install gradio